# Pengenalan kepada Kejuruteraan Prompt
Kejuruteraan prompt ialah proses mereka bentuk dan mengoptimumkan prompt untuk tugasan pemprosesan bahasa semula jadi. Ia melibatkan pemilihan prompt yang sesuai, melaras parameter mereka, dan menilai prestasi mereka. Kejuruteraan prompt sangat penting untuk mencapai ketepatan dan kecekapan yang tinggi dalam model NLP. Dalam bahagian ini, kita akan meneroka asas-asas kejuruteraan prompt menggunakan model OpenAI untuk tujuan penerokaan.


### Latihan 1: Tokenisasi
Terokai Tokenisasi menggunakan tiktoken, penanda token pantas sumber terbuka dari OpenAI
Lihat [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst) untuk lebih banyak contoh.


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### Latihan 2: Sahkan Persediaan Kunci Github Models

Jalankan kod di bawah untuk memastikan bahawa endpoint Github Models anda telah disediakan dengan betul. Kod ini hanya mencuba prompt asas yang ringkas dan mengesahkan hasilnya. Input `oh say can you see` sepatutnya dilengkapkan dengan sesuatu seperti `by the dawn's early light..`


In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt, client, model_name)
print(response)


That line is the opening lyric of "The Star-Spangled Banner," the national anthem of the United States, written by Francis Scott Key. If you'd like more information or analysis, feel free to ask!


### Latihan 3: Rekaan

Terokai apa yang berlaku apabila anda meminta LLM memberikan jawapan untuk arahan mengenai topik yang mungkin tidak wujud, atau tentang topik yang mungkin tidak diketahui olehnya kerana berada di luar set data pra-latihannya (lebih terkini). Lihat bagaimana respons berubah jika anda mencuba arahan yang berbeza, atau model yang berbeza.


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

### Latihan 4: Berdasarkan Arahan
Gunakan pembolehubah "text" untuk menetapkan kandungan utama 
dan pembolehubah "prompt" untuk memberikan arahan berkaitan dengan kandungan utama itu.

Di sini kita minta model untuk meringkaskan teks untuk pelajar darjah dua


In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt, client, model_name)
print(response)

Jupiter is the fifth planet from the Sun and the biggest one in our Solar System. It's made of gas and is much bigger than all the other planets put together! You can see Jupiter in the night sky because it's very bright. People have noticed it for a really long time and named it after a Roman god.


### Latihan 5: Arahan Kompleks
Cuba satu permintaan yang mempunyai mesej sistem, pengguna dan pembantu
Sistem menetapkan konteks pembantu
Mesej Pengguna & Pembantu memberikan konteks perbualan berbilang pusingan

Perhatikan bagaimana personaliti pembantu ditetapkan kepada "sinis" dalam konteks sistem.
Cuba gunakan konteks personaliti yang berbeza. Atau cuba siri mesej input/output yang berbeza


In [5]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, you mean the famous 2020 World Series that wasn’t in a regular location? That was the year they played in the glamorous Arlington, Texas, at Globe Life Field.


### Latihan: Terokai Intuisi Anda
Contoh-contoh di atas memberikan anda corak yang boleh digunakan untuk mencipta prompt baru (mudah, kompleks, arahan dan sebagainya) - cuba cipta latihan lain untuk meneroka idea-idea lain yang telah kita bincangkan seperti contoh, petunjuk dan banyak lagi.



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila maklum bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang sah. Untuk maklumat penting, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
